# Problem 1: Generating Random Boolean Functions

The Deutsch–Jozsa algorithm is designed to work with functions that accept a fixed number of Boolean inputs and return a single Boolean output. Each function is guaranteed to be either constant (always returns False or always returns True) or balanced (returns True for exactly half of the possible input combinations).

**Purpose**: Generate a random boolean function with 4 inputs that is constant or balanced

**Fornula**:
Constant: `f(a,b,c,d) = k` where k ∈ {True, False} for all inputs
Balanced: `f(a,b,c,d) = True` for exactly 8 of 16 possible inputs

**Behaviour**: Returns a callable function that accepts 4 Boolean arguments and returns a boolean

## Solution
Will generate all 16 possible 4-bit Boolean input combinations using `itertools.product([False, True], repeat=4)`. The function will randomly choose between creating a constant or a balanced function. For the constantfunction, all inputs map to the same Boolean value. While for the balanced function, will use the `random.sample()` to randomly select 8 of the 16 inputs to return True, with the remaining 8 inputs returning False. The dictionary will store the input/output mapping, and then will return a closure function that performs O(1) lookups on the dictionary to determine the output for any input combination.

## Operations
`itertools.product([False, True], repeat=4)` generates all input combinations
`random.choice()` selects function type and constant value
`random.sample()` selects 8 random inputs for balanced functions
Dictionary comprehension creates the function mapping efficiently

## References
IBM Quantum Learning - Deutsch-Jozsa Algorithm: https://quantum.cloud.ibm.com/learning/en/modules/computer-science/deutsch-jozsa
Python itertools documentation: https://docs.python.org/3/library/itertools.html
Nielsen & Chuang, "Quantum Computation and Quantum Information", Section 1.4.3

In [1]:
# All the neccessary imports for my notebook
import random
from itertools import product
from typing import Callable
import numpy as np
import matplotlib.pyplot as plt

## Helper Function: generate_inputs()

## Purpose
The helper function generates all the possible boolean input combinations for a given number of bits. It's used to create the complete input space for testing Boolean functions.

### Implementation
The helper function uses Python's 'itertools.product()' to create a Cartesian product of '[False, True]' repeated 'n_bits' times. This generates all the possible combinations without manual iteration.

### Example
For 4 bits:
- Total combinations: $2^4 = 16$
- Output ranges from '(False, False, False, False)' to '(True, True, True, True)'

In [2]:
# Helper function 
# Generates all possible input combinations
# Uses itertools.product to create a cartesian product of False or True repeated n times
# See https://docs.python.org/3/library/itertools.html#itertools.product
def generate_inputs(n_bits: int) -> list:
    """
    Generate all possible boolean inputs for n bits

    The function creates all possible tuples of boolean values of length n_bits
    
    n_bits: number of boolean input bits
    returns a list of tuples representing all possible input combinations
    """
    # Using itertools.product to generate all combinations
    # Product([False, True], repeat=n_bits) creates a cartesian product
    # Converting to list for easier indexing and manipulation
    return list(product([False, True], repeat=n_bits))

# Testing the helper function
test_inputs = generate_inputs(4)
print(f"Number of possible inputs for 4 bits: {len(test_inputs)}")
print(f"First 5 input combinations: {test_inputs[:5]}")
print(f"Last 5 input combinations: {test_inputs[-5:]}")

Number of possible inputs for 4 bits: 16
First 5 input combinations: [(False, False, False, False), (False, False, False, True), (False, False, True, False), (False, False, True, True), (False, True, False, False)]
Last 5 input combinations: [(True, False, True, True), (True, True, False, False), (True, True, False, True), (True, True, True, False), (True, True, True, True)]


In [3]:
# Generates a random constant or balanced Boolean function with 4 inputs
# Returns a callable function that maps 4 Boolean inputs to a Boolean output
# See https://quantum.cloud.ibm.com/learning/en/modules/computer-science/deutsch-jozsa
def random_constant_balanced() -> Callable:
    """
    Generating a random constant or balanced Boolean function with 4 inputs
    Formula: Constant returns same value for all 16 inputs, Balanced returns True for exactly 8/16 inputs
    
    The function randomly chooses between constant or balanced type
    For constant all 16 inputs map to same Boolean value (True or False)
    For balanced exactly 8 inputs map to True and the remaining 8 inputs map to False
    
    returns a callable function f(a,b,c,d) that accepts 4 Boolean arguments and returns Boolean output
    """
    # Generating all 16 possible input combinations using the helper function
    all_inputs = generate_inputs(4)
    
    # Randomly selects function type
    # random.choice() picks either constant or balanced
    function_type = random.choice(['constant', 'balanced'])
    
    # if random.choice picks constant all inputs map to the same value
    if function_type == 'constant':
        # Randomly choose True or False as the constant output
        constant_value = random.choice([True, False])
        # Dictionary comprehension maps all inputs to same constant value
        function_map = {inputs: constant_value for inputs in all_inputs}
        
    # if random.choice picks balanced exactly 8 inputs return true and the other 8 return false
    else:
        # random.sample() selects 8 unique random inputs without replacement
        true_inputs = random.sample(all_inputs, 8)
        # Dictionary comprehension maps selected inputs to True and the other inputs to False
        function_map = {inputs: (inputs in true_inputs) for inputs in all_inputs}
    
    # Returning a closure function that encapsulates the mapping
    def boolean_function(a: bool, b: bool, c: bool, d: bool) -> bool:
        """
        Boolean function with 4 inputs
        Performs O(1) dictionary lookup to determine output
        
        a, b, c, d: Boolean input values
        returns a boolean output based on the generated function mapping
        """
        # Dictionary lookup using input tuple as key
        return function_map[(a, b, c, d)]
    
    # Stores metadata for testing
    boolean_function.function_type = function_type
    boolean_function.function_map = function_map
    
    return boolean_function